# Data Pipeline
For this project determining which strategies to use determines the sucess of the trading bot.<br>
In the context of this project we follow this structure in 3 steps found in these folders in `src/`<br>
`backtesting/` -> `data/` -> `models/`<br>
1. `backtesting/` - Here we try out any choice of strategies we want to try using `backtrader` to find a viable technical indicator based approach to ensure that historically our selection would turn a profit.
2. `data/` - Here we then implement those technical indicators onto a `DataFrame` from historical data on a stock of our choice. This also includes the signals to indicate that we should buy or sell, this makes the `DataFrame` contain all the features and labels necessary to train model on.
3. `models/` - Here we train the machine learning models on the modified `DataFrame`s and validate that they can then accurately predict the buy and sell signals based on technical indicators<br>
Given this pipeline we can ensure the validity of the technical indicators and allow machine learning models to train on them. 

# Backtesting
We first test the strategy on historical data using technicals of our choice using `backtrader`, once we decide what technicals indicators we would like to use we then apply them in the `process_data()` method found in `src/data/data_processing.py`. After we define the modifications to the `DataFrame` then we can start training the ML models

In [2]:
import sys
import os
import sys
sys.path.insert(0, '../')

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from data import data_processing as dp # to get modified DataFrames with Technicals

# Get DataFrame
In `src/data/data_processing.py` we can load a `DataFrame` that contains our own technical indicators using the `get_df` method and a spcecified ticker. We can modify the strategy and technical indicators in the `process_data` method to adjust to new strategies. This method will also add the signals to the df to allow ML models to train on. 

In [5]:
df = dp.get_df("AAPL")
df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,SMA(10),SMA(30),EMA(12),EMA(26),EMA(12-26),Signal Line(9),SMA Crossover,MACD,Signal
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,,,,,,,,,
Date,,,,,,,,,,,,,,
1981-01-26,0.110745,0.111174,0.110745,0.111174,24640000,0.109371,0.107526,0.110262,0.108431,0.001831,0.001952,0,0,HOLD
1981-01-27,0.109886,0.110745,0.109886,0.110745,23699200,0.109886,0.107898,0.110204,0.108539,0.001665,0.001895,0,0,HOLD
1981-01-28,0.106453,0.106882,0.106453,0.106882,28156800,0.110015,0.108327,0.109627,0.108384,0.001243,0.001764,0,0,HOLD
1981-01-29,0.102590,0.103019,0.102590,0.103019,43904000,0.109543,0.108856,0.108544,0.107955,0.000589,0.001529,0,0,HOLD
1981-01-30,0.097009,0.097868,0.097009,0.097868,46188800,0.108599,0.109128,0.106770,0.107144,-0.000375,0.001149,-1,0,HOLD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-06,198.509995,200.649994,197.020004,198.210007,51216500,207.217001,205.003001,204.458987,206.220323,-1.761336,-2.454296,0,0,HOLD
2025-05-07,196.250000,199.440002,193.250000,199.169998,68536700,206.382001,204.086335,203.196066,205.481780,-2.285714,-2.420580,0,0,HOLD


# Construct ML model
Once we have a dataframe with the technical indicators we would like to use, we can construct the ML model. We will use scikit-learn to simplify the process. Here we will scale the data, set up the pipeline, and split the data up. However most of the signals in the `DataFrame` are just "HOLD" so we need to do a lot preprocessing

In [69]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression # Various ML models 
from sklearn.preprocessing import StandardScaler, QuantileTransformer # Import for scaling the data
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
    ("scale", StandardScaler()),
    ("model", LinearRegression())
])






# Train ML Model
We will test the model on the training data which will be a chunk of the dataframe we got from yfinance. Then once trained we can put it another chunk for validation data and adjust parameters as needed. Finally we can run the model on the test data for final analysis. 

# Export Model 
Once the model is trained we can export it using the `pickle` library or another equivalent. We can then use this in a driver where 